In [2]:
import torch

In [3]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Harry         (x^1)
     [0.55, 0.87, 0.66], # Potter        (x^2)
     [0.57, 0.85, 0.64], # and           (x^3)
     [0.22, 0.58, 0.33], # the           (x^4)
     [0.77, 0.25, 0.10], # Philosopher's (x^5)
     [0.05, 0.80, 0.55]] # Stone         (x^6)
)

In [4]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [5]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [6]:
(0.55*0.43) + (0.87*0.15) + (0.66*0.89)

0.9544

In [7]:
attention_score_21 = torch.dot(input_query, input_1)
attention_score_21

tensor(0.9544)

In [8]:
i = 1
attention = torch.dot(input_query, inputs[i])
attention

tensor(1.4950)

In [9]:
attention_scores_input_2 = torch.empty(inputs.shape[0])
for idx, item in enumerate(inputs):
    attention_scores_input_2[idx] = torch.dot(input_query, inputs[idx])

attention_scores_input_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [10]:
attention_weights_temp = attention_scores_input_2 / attention_scores_input_2.sum()
attention_weights_temp

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [11]:
attention_weights_temp.sum()

tensor(1.0000)

In [12]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attention_weights_2 = softmax_naive(attention_scores_input_2)
attention_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [13]:
attention_weights_2.sum()

tensor(1.)

In [14]:
context_vector_2 = torch.zeros(inputs[1].shape[0])

for i, item in enumerate(inputs):
    context_vector_2 += attention_weights_2[i]*inputs[i]

context_vector_2

tensor([0.4419, 0.6515, 0.5683])

In [15]:
all_attention_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        all_attention_scores[i,j] = torch.dot(x_i, x_j)

all_attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [16]:
all_attention_scores = inputs @ inputs.T
all_attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [17]:
all_attention_weights = torch.softmax(all_attention_scores , dim=1)
all_attention_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [18]:
all_attention_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [19]:
all_context_vectors = all_attention_weights @ inputs
all_context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## Self attention with trainable weights

In [20]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [21]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [22]:
querys = inputs @ W_query
keys = inputs @ W_key
values = inputs @ W_value
querys

tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]], grad_fn=<MmBackward0>)

In [23]:
query_2 = x_2 @ W_query
query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [24]:
key_2 = keys[1]
attention_scores_22 = torch.dot(query_2, key_2)
attention_scores_22
# attention_scores = query_2 @ keys.T

tensor(1.8524, grad_fn=<DotBackward0>)

In [27]:
attention_scores_2 = query_2 @ keys.T
attention_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [30]:
d_k = keys.shape[1]
attention_weights_2 = torch.softmax(attention_scores_2/ d_k**0.5, dim=-1)
attention_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)

In [31]:
context_vector_2 = attention_weights_2 @ values
context_vector_2

tensor([0.3061, 0.8210], grad_fn=<SqueezeBackward4>)

# Implementing a compact Self-Attention class

In [32]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        querys = x @ self.W_query
        keys = x @ self.W_key
        values = x @ self.W_value

        attention_scores = querys @ keys.T
        attention_weights = torch.softmax(attention_scores /  d_k**0.5, dim=-1)

        context_vectors = attention_weights @ values

        return context_vectors
        

torch.manual_seed(123)
self_attention_v1 = SelfAttention_v1(d_in, d_out)
self_attention_v1(inputs)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

In [33]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        querys = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_scores = querys @ keys.T
        attention_weights = torch.softmax(attention_scores /  d_k**0.5, dim=-1)

        context_vectors = attention_weights @ values

        return context_vectors
        

torch.manual_seed(123)
self_attention_v2 = SelfAttention_v2(d_in, d_out)
self_attention_v2(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

# Causal Attention

In [34]:
querys = self_attention_v2.W_query(inputs)
keys = self_attention_v2.W_key(inputs)
values = self_attention_v2.W_value(inputs)

attention_scores = querys @ keys.T
attention_weights = torch.softmax(attention_scores /  d_k**0.5, dim=-1)
attention_weights
attention_scores

tensor([[0.3111, 0.3479, 0.3471, 0.1714, 0.2350, 0.1928],
        [0.1655, 0.2602, 0.2576, 0.1445, 0.1384, 0.1790],
        [0.1667, 0.2602, 0.2577, 0.1443, 0.1391, 0.1784],
        [0.0510, 0.1080, 0.1064, 0.0643, 0.0476, 0.0835],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121, 0.1174],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MmBackward0>)

In [35]:
context_length = attention_scores.shape[0]
context_length

6

In [36]:
mask_simple = torch.tril(torch.ones(context_length, context_length))

In [37]:
masked_weights = attention_weights * mask_simple

In [38]:
rows_sum = masked_weights.sum(dim=-1, keepdim=True)
normalized_masked_weights = masked_weights / rows_sum
normalized_masked_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)

In [39]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked_attention_scores = attention_scores.masked_fill(mask.bool(), -torch.inf)
masked_attention_scores

tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)

In [40]:
masked_attention_weights = torch.softmax(masked_attention_scores /  keys.shape[1]**0.5, dim=-1)
masked_attention_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)

# Masking additional attentional weights with dropout

In [41]:
torch.manual_seed(123)
dropout_layer = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
example

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [42]:
dropout_layer(example)

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])

In [43]:
dropout_layer(masked_attention_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6816, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5085, 0.4936, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3906, 0.0000],
        [0.3249, 0.3418, 0.0000, 0.3308, 0.3249, 0.3363]],
       grad_fn=<MulBackward0>)

# Compact Causal self attention class

In [44]:
batch = torch.stack((inputs, inputs), dim=0)
batch.shape

torch.Size([2, 6, 3])

In [45]:
import torch.nn as nn

class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout_layer = torch.nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
        

    def forward(self, x):
        batch_size, num_tokens, d_in = x.shape
        querys = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_scores = querys @ keys.transpose(1,2)
        attention_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attention_weights = torch.softmax(attention_scores /  keys.shape[-1]**0.5, dim=-1)
        attention_weights = self.dropout_layer(attention_weights)

        context_vectors = attention_weights @ values

        return context_vectors

In [46]:
torch.manual_seed(789)
context_length = batch.shape[1]
dropout = 0.0
causal_attention = CausalAttention(d_in, d_out,context_length, dropout)
causal_attention(batch)

tensor([[[-0.0872,  0.0286],
         [-0.0991,  0.0501],
         [-0.0999,  0.0633],
         [-0.0983,  0.0489],
         [-0.0514,  0.1098],
         [-0.0754,  0.0693]],

        [[-0.0872,  0.0286],
         [-0.0991,  0.0501],
         [-0.0999,  0.0633],
         [-0.0983,  0.0489],
         [-0.0514,  0.1098],
         [-0.0754,  0.0693]]], grad_fn=<UnsafeViewBackward0>)

# Extending single head attention to multi head attention

In [49]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out,context_length, dropout, num_heads=2, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([CausalAttention(d_in, d_out,context_length, dropout, qkv_bias) for _ in range(num_heads)])


    def forward(self, x):
      return torch.cat([head(x)  for head in self.heads], dim=-1)


torch.manual_seed(123)
context_length = batch.shape[1]
d_in = batch.shape[2]
d_out = 2
dropout = 0.0

multi_head_attention_stacked = MultiHeadAttentionWrapper(d_in, d_out,context_length, dropout, num_heads=2)
multi_head_attention_stacked(batch)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)

# Multihead Attention with weight splits

In [56]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out,context_length, dropout, num_heads=2, qkv_bias=False):
        super().__init__()
        assert(d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out)
        self.dropout_layer = torch.nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))


    def forward(self, x):
        batch_size, num_tokens, d_in = x.shape

        queries = self.W_query(x)  #(batch_size, num_tokens, d_out)
        keys = self.W_key(x)
        values = self.W_value(x)

        queries = queries.view(batch_size, num_tokens, self.num_heads, self.head_dim)   #(batch_size, num_tokens, num_heads, head_dim)
        keys = keys.view(batch_size, num_tokens, self.num_heads, self.head_dim)
        values = values.view(batch_size, num_tokens, self.num_heads, self.head_dim)

        queries = queries.transpose(1, 2)   #(batch_size, num_heads, num_tokens, head_dim)
        print(queries.shape)
        keys = keys.transpose(1, 2)
        values = values.transpose(1, 2)

        attention_scores = queries @ keys.transpose(2, 3) #dot product for each head
        print(attention_scores.shape)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attention_scores.masked_fill_(mask_bool, -torch.inf)

        attention_weights = torch.softmax(attention_scores /  keys.shape[-1]**0.5, dim=-1)
        attention_weights = self.dropout_layer(attention_weights)
        print(attention_weights.shape)

        context_vectors = (attention_weights @ values).transpose(1,2) #(batch_size, num_tokens, num_heads, head_dim)
        print(context_vectors.shape)

        
        # self.d_out = self.num_heads * self.head_dim
        context_vectors = context_vectors.contiguous().view(batch_size, num_tokens, self.d_out)
        print(context_vectors.shape)
        context_vectors = self.out_proj(context_vectors) #optional projection
        print(context_vectors.shape)

        return context_vectors

torch.manual_seed(123)
context_length = batch.shape[1]
d_in = batch.shape[2]
d_out = 4
dropout = 0.0

multi_head_attention_stacked = MultiHeadAttention(d_in, d_out,context_length, dropout, num_heads=2)
multi_head_attention_stacked(batch)
        

torch.Size([2, 2, 6, 2])
torch.Size([2, 2, 6, 6])
torch.Size([2, 2, 6, 6])
torch.Size([2, 6, 2, 2])
torch.Size([2, 6, 4])
torch.Size([2, 6, 4])


tensor([[[-0.3132, -0.2272,  0.4772,  0.1063],
         [-0.2308,  0.0329,  0.5764,  0.3007],
         [-0.2059,  0.1190,  0.6097,  0.3654],
         [-0.1642,  0.1340,  0.5431,  0.3503],
         [-0.1689,  0.1794,  0.5296,  0.3389],
         [-0.1407,  0.1699,  0.5040,  0.3403]],

        [[-0.3132, -0.2272,  0.4772,  0.1063],
         [-0.2308,  0.0329,  0.5764,  0.3007],
         [-0.2059,  0.1190,  0.6097,  0.3654],
         [-0.1642,  0.1340,  0.5431,  0.3503],
         [-0.1689,  0.1794,  0.5296,  0.3389],
         [-0.1407,  0.1699,  0.5040,  0.3403]]], grad_fn=<ViewBackward0>)